# 特征工程

## 特征选择
特征选择是特征工程的第一步，它关系到我们机器学习算法的上限。因此原则是**尽量不错过一个可能有用的特征**，但是也**不滥用太多的特征**。
### 过滤法
#### 方差过滤
  指定一个方差的阈值，当方差小于这个阈值的特征会被我们筛掉。sklearn中的VarianceThreshold类可实现这一操作

In [1]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold
data = np.array([[1,2,3],[1,1,3],[4,5,6]])
vt = VarianceThreshold(threshold=2)
vt.fit_transform(data)

array([[2],
       [1],
       [5]])

#### 相关系数
分别计算所有训练集中各个特征与输出值之间的相关系数，设定一个阈值，选择相关系数较大的部分特征。 

In [7]:
x = np.array([1,1,4])
y = np.array([1,2,1])
np.corrcoef(x.T, y.T)

array([[ 1. , -0.5],
       [-0.5,  1. ]])

In [4]:
from sklearn.feature_selection import SelectKBest
import numpy as np
X = np.array([[1,2,3],[1,1,3],[4,5,6]])
y = np.array([1,2,1])
X_corr = SelectKBest(lambda X, y: 
                     np.array(list(map(lambda x: np.corrcoef(x.T, y.T)[0,1], X.T))),
                     k=2).fit_transform(X, y)

In [5]:
X_corr

array([[1, 3],
       [1, 3],
       [4, 6]])

(注： `np.corrcoef`返回的是矩阵)

#### 假设检验
_卡方检验_ 可以检验某个特征分布和输出值分布之间的相关性。在sklearn中，可以使用**chi2**这个类来做卡方检验得到所有特征的卡方值与显著性水平P临界值，我们可以给定卡方值阈值， 选择卡方值较大的部分特征。卡方检验原理可参见[此文](https://segmentfault.com/a/1190000003719712)。

In [9]:
from sklearn.feature_selection import SelectKBest ,chi2
X = np.array([[1,2,3],[1,1,3],[4,5,6]])
y = [1,2,1]
X_chi2 = SelectKBest(chi2, k=2).fit_transform(X, y)
X_chi2

array([[1, 2],
       [1, 1],
       [4, 5]])


除此之外还有 _f-检验_ 及 _t-检验_

#### 互信息
互信息值越大，说明该特征和输出值之间的相关性越大，越需要保留。在sklearn中，可以使用**mutual_info_classif**(分类)和**mutual_info_regression**(回归)来计算各个输入特征和输出值之间的互信息。关于互信息详细介绍可查看机器学习决策树相关理论。

In [10]:
from sklearn.feature_selection import SelectKBest
from minepy import MINE
def mic(x, y):
    m = MINE()
    m.compute_score(x, y) 
    return m.mic()

X_mutal = SelectKBest(lambda X, Y: np.array(list(
    map(lambda x:mic(x, Y), X.T))), k=2).fit_transform(X, y)

X_mutal

ModuleNotFoundError: No module named 'minepy'

以上就是过滤法的主要方法，在没有思路时可选取卡方检验及互信息。



### 包装法
最常用的包装法是递归消除特征法(recursive feature elimination,以下简称RFE)。递归消除特征法使用一个机器学习模型来进行多轮训练，每轮训练后，消除若干权值系数的对应的特征，再基于新的特征集进行下一轮训练。在sklearn中，可以使用RFE函数来选择特征。
### 嵌入法选择特征
嵌入法也是用机器学习的方法来选择特征，但是它和RFE的区别是它不是通过不停的筛掉特征来进行训练，而是使用的都是特征全集。在sklearn中，使用SelectFromModel函数来选择特征。最常用的是使用L1正则化和L2正则化来选择特征。  
最常用的是使用L1正则化和L2正则化来选择特征。在正则化模型中，正则化惩罚项越大，那么模型的系数就会越小。当正则化惩罚项大到一定的程度的时候，部分特征系数会变成0，当正则化惩罚项继续增大到一定程度时，所有的特征系数都会趋于0. 但是我们会发现一部分特征系数会更容易先变成0，这部分系数就是可以筛掉的。也就是说，我们选择特征系数较大的特征。常用的L1正则化和L2正则化来选择特征的基学习器是逻辑回归。
### 寻找高级特征
在我们拿到已有的特征后，我们还可以根据需要寻找到更多的高级特征。比如有车的路程特征和时间间隔特征，我们就可以得到车的平均速度这个二级特征。根据车的速度特征，我们就可以得到车的加速度这个三级特征，根据车的加速度特征，我们就可以得到车的加加速度这个四级特征。。。也就是说，高级特征可以一直寻找下去。  
在Kaggle之类的算法竞赛中，高分团队主要使用的方法除了**集成学习算法**，剩下的主要就是**在高级特征上面做文章**。所以寻找高级特征是模型优化的必要步骤之一。  
寻找高级特征最常用的方法有：
1. 若干项特征加和：我们假设你希望根据每日销售额得到一周销售额的特征。你可以将最近的7天的销售额相加得到。
2. 若干项特征之差：假设你已经拥有每周销售额以及每月销售额两项特征，可以求一周前一月内的销售额。
3. 若干项特征乘积：假设你有商品价格和商品销量的特征，那么就可以得到销售额的特征。
4. 若干项特征除商：假设你有每个用户的销售额和购买的商品件数，那么就是得到该用户平均每件商品的销售额。

## 特征表达
### 缺失值处理
特征有缺失值是非常常见的，大部分机器学习模型在拟合前需要所有的特征都有值，不能是空或者NULL。如果是连续值，那么一般有两种选择，一是选择所有有该特征值的样本，然后取平均值，来填充缺失值，另一种是取中位数来填充缺失值。如果是离散值，则一般会选择所有有该特征值的样本中最频繁出现的类别值，来填充缺失值。在sklearn中，可以使用**preprocessing.Imputer**来选择这三种不同的处理逻辑做预处理。
```
import numpy as np
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0) (注：missing_values用于表征缺失值）
X = np.array([[1,2,3],['NAN',1,2],[1,2,2], [2,'Nan','nan']])
x = imp.fit_transform(X)
```
### 特殊的特征处理
有些特征的默认取值比较特殊，一般需要做了处理后才能用于算法。比如日期时间，地理特征等。
### 离散特征的连续化处理
有很多机器学习算法只能处理连续值特征，不能处理离散值特征，比如线性回归，逻辑回归等。  
最常见的离散特征连续化的处理方法是独热编码**one-hot encoding**。处理方法其实比较简单，比如某特征的取值是高，中和低，那么我们就可以创建三个取值为0或者1的特征，将高编码为1,0,0这样三个特征，中编码为0,1,0这样三个特征，低编码为0,0,1这样三个特征。也就是说，之前的一个特征被我们转化为了三个特征。sklearn的**OneHotEncoder**可以帮我们做这个处理。此外还有**LabelEncoder**，但是该方法产生的特征取值的大小会对优化结果有一定的影响。
```
import numpy as np
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder(categorical_features=[0,1],dtype=int)
X = np.array([[1,2,3],
              [4,1,2],
              [1,2,2], 
              [2,0,2]])
x = oh.fit_transform(X).toarray()
```
```
import numpy as np
from sklearn.preprocessing import LabelEncoder
X = np.array([[1,'a',3],
              [4,'b',2],
              [1,'l',2], 
              [2,'m',2]])
le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])
```
### 离散特征的离散化处理
离散特征有时间也不能直接使用，需要先进行转化。比如最常见的，如果特征的取值是高，中和低，那么就算你需要的是离散值，也是没法直接使用的。对于原始的离散值特征，最常用的方法也是独热编码，第二种方法是虚拟编码dummy coding，它和独热编码类似。
### 连续特征的离散化处理
对于连续特征，有时候我们也可以将其做离散化处理。这样特征变得高维稀疏，方便一些算法的处理。常用的方法是根据阈值进行分组，比如我们根据连续值特征的分位数，将该特征分为高，中和低三个特征。将分位数从0-0.3的设置为高，0.3-0.7的设置为中，0.7-1的设置为高。
## 特征预处理
### 特征的标准化和归一化
虽然大部分机器学习模型都需要做标准化和归一化，也有不少模型可以不做做标准化和归一化，主要是基于概率分布的模型，比如决策树大家族的CART，随机森林等。当然此时使用标准化也是可以的，大多数情况下对模型的泛化能力也有改进。
#### z-score标准化
具体的方法是求出样本特征x的均值mean和标准差std，然后用**（x-mean)/std**来代替原特征。这样特征就变成了均值为0，方差为1了。在sklearn中，我们可以用**StandardScaler**来做z-score标准化。
```
import numpy as np
from sklearn.preprocessing import StandardScaler
X = np.array([[1,2,3],
              [4,22,2],
              [1,11,2], 
              [2,6,2]])
std = StandardScaler()
X = std.fit_transform(X)
```
#### max-min标准化
也称为离差标准化，预处理后使特征值映射到[0,1]之间。具体的方法是求出样本特征x的最大值max和最小值min，征然后用`(x-min)/(max-min)`来代替原特征。如果我们希望将数据映射到任意一个区间[a,b]，而不是[0,1]，那么也很简单。用`(x-min)(b-a)/(max-min)+a`来代替原特征即可。在sklearn中，我们可以用**MinMaxScaler**来做max-min标准化。这种方法的问题就是如果测试集或者预测数据里的特征有小于min，或者大于max的数据，会导致max和min发生变化，需要重新计算。所以实际算法中， 除非你对特征的取值区间有需求，否则max-min标准化没有z-score标准化好用。
#### L1/L2范数标准化
如果我们只是为了统一量纲，那么通过L2范数整体标准化也是可以的，具体方法是求出每个样本特征向量x的L2范数,然后用**x/L2**代替原样本特征即可。当然L1范数标准化也是可以的，即用x/L1代替原样本特征。通常情况下，范数标准化首选L2范数标准化。在sklearn中，我们可以用**Normalizer**来做L1/L2范数标准化。
```
import numpy as np
from sklearn.preprocessing import Normalizer
X = np.array([[1,2,3],
              [4,22,2],
              [1,11,2], 
              [2,6,2]])
nm = Normalizer()
X = nm.fit_transform(X)
```
#### 中心化
此外，经常我们还会用到中心化，主要是在PCA降维的时候，此时我们求出特征x的平均值mean后，用**x-mean**代替原特征，也就是特征的均值变成了0, 但是方差并不改变。这个很好理解，因为PCA就是依赖方差来降维的，如果我们做了z-score标准化，所以特征的方差为1，那么就没法来降维了。
### 异常特征样本清洗
我们在实际项目中拿到的数据往往有不少异常数据，有时候不筛选出这些异常数据很可能让我们后面的数据分析模型有很大的偏差。那么如果我们没有专业知识，如何筛选出这些异常特征样本呢？常用的方法有两种。  
第一种是**聚类**，比如我们可以用KMeans聚类将训练样本分成若干个簇，如果某一个簇里的样本数很少，而且簇质心和其他所有的簇都很远，那么这个簇里面的样本极有可能是异常特征样本了。我们可以将其从训练集过滤掉。  
第二种是异常点检测方法，主要是使用iForest或者one class SVM，使用异常点检测的机器学习算法来过滤所有的异常点。  
当然，某些筛选出来的异常样本是否真的是不需要的异常特征样本，最好找懂业务的再确认一下，防止我们将正常的样本过滤掉了。
### 处理不平衡数据
对于类别不均衡问题，小类别数据的预测准确一般较低。为解决这个问题，一般可采用两种方法：**权重法**或者**采样法**。  
权重法是比较简单的方法，我们可以对训练集里的每个类别加一个权重**class weight**。如果该类别的样本数多，那么它的权重就低，反之则权重就高。如果更细致点，我们还可以对每个样本加权重**sample weight**，思路和类别权重也是一样，即样本数多的类别样本权重低，反之样本权重高。sklearn中，绝大多数分类算法都有class weight和 sample weight可以使用。  
如果权重法做了以后发现预测效果还不好，可以考虑采样法。  
采样法常用的也有两种思路，一种是对类别样本数多的样本做**欠采样**, 比如训练集里A类别样本占90%，B类别样本占10%。那么我们可以对A类的样本子采样，直到子采样得到的A类样本数和B类别现有样本一致为止，这样我们就只用子采样得到的A类样本数和B类现有样本一起做训练集拟合模型。第二种思路是对类别样本数少的样本做**过采样**, 还是上面的例子，我们对B类别的样本做过采样，直到过采样得到的B类别样本数加上B类别原来样本一起和A类样本数一致，最后再去拟合模型。  
上述两种常用的采样法很简单，但是都有个问题，就是**采样后改变了训练集的分布，可能导致泛化能力差**。所以有的算法就通过其他方法来避免这个问题，比如**SMOTE算法**通过人工合成的方法来生成少类别的样本。方法也很简单，对于某一个缺少样本的类别，它会随机找出几个该类别的样本，再找出最靠近这些样本的若干个该类别样本，组成一个候选合成集合，然后在这个集合中不停的选择距离较近的两个样本，在这两个样本之间，比如中点，构造一个新的该类别样本。举个例子，比如该类别的候选合成集合有两个样本(x1,y),(x2,y),那么SMOTE采样后，可以得到一个新的训练样本(x1+x22,y),通过这种方法，我们可以得到不改变训练集分布的新样本，让训练集中各个类别的样本数趋于平衡。我们可以用**imbalance-learn**这个Python库中的**SMOTEENN**类来做SMOTE采样。

In [34]:
import numpy as np
from sklearn.preprocessing import Normalizer
X = np.array([[1,2,3],
              [4,22,2],
              [1,11,2], 
              [2,6,2]])
nm = Normalizer()
X = nm.fit_transform(X)
X

array([[0.26726124, 0.53452248, 0.80178373],
       [0.17817416, 0.97995789, 0.08908708],
       [0.08908708, 0.97995789, 0.17817416],
       [0.30151134, 0.90453403, 0.30151134]])

In [27]:
a = ['a','b','l','m']
a1 = le.fit_transform(a)
a1

array([0, 1, 2, 3])